In [105]:
import fastai
import torch
import torch.nn as nn
from torch.nn import init
import numpy as np
from nltk.tokenize import word_tokenize
import nltk
import json 
import os
import pickle
import requests
from torch.nn.utils.rnn import pad_sequence
# # Loading all training examples, validation examples, and vocabulary

# with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Paraphrase_Generator/quora_raw_train.json') as f:
#     training_examples = json.load(f)
    
# with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Paraphrase_Generator/quora_raw_val.json') as f:
#     val_examples = json.load(f)

# with open('/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Paraphrase_Generator/quora_data_prepro.json') as f:
#     vocab = json.load(f)['ix_to_word'] # Vocab = {1: 'the', 2: 'sauce' .....}
# vocab_inv = {word:number for number,word in list(vocab.items())} # Inverted dictionary created   Vocab_inv = {'the':1, 'sauce':2, .....}

# X_train_prelim = [training_examples[i]['question1'] for i in range(len(training_examples))] # Training examples before preprocessing
# Y_train_prelim = [training_examples[i]['question'] for i in range(len(training_examples))]

# X_val_prelim = [val_examples[i]['question1'] for i in range(len(val_examples))]
# Y_val_prelim = [val_examples[i]['question'] for i in range(len(val_examples))]

# def load_glove_embeddings(filename="glove.6B.100d.txt"):
#     try:
#         with open("/Users/varunbabbar/Desktop/Good_Quotes/Paraphrase_Generator/Paraphrase_Generator/embeddings", "rb") as f:
#             embeddings = pickle.load(f)
#         return embeddings
#     except:
#         lines = open(filename).readlines()
#         print(len(lines))
#         embeddings = {}
#         a = 0
#         for line in lines:
#             a +=1
#             print(a)
#             word = line.split()[0]
#             embedding = list(map(float, line.split()[1:]))
#             if word in vocab.values():
#                 embeddings[int(vocab_inv[word])] = embedding # This is creating a dictionary with indexes corresponding to the position of the particular word in the vocabulary
#                 # embeddings = {1:embedding of 'the', 2: embedding of 'sauce'....}
#         return embeddings

# def tokenize_embed(X,Y): # Doing this for training and validation_datasets
#     X_train = []
#     Y_train = []
#     for i in range(len(X)): # This is basically tokenising each sentence, and converting it into a list of word embeddings. X corresponds to a training example and Y corresponds to its paraphrase
#         print(i)
#         sentence_x = X[i]
#         sentence_y = Y[i]
#         tokensx = word_tokenize(sentence_x.lower())
#         tokensy = word_tokenize(sentence_y.lower())
#         vectorx = []
#         vectory = []
#         for word in tokensx:
#             try:
#                 vectorx.append(int(vocab_inv[word]))
#             except:
#                 vectorx.append(258) # If the word is not in the embeddings given, then add a ',' instead. (Need to discuss this)
#         for word in tokensy:
#             try:
#                 vectory.append(int(vocab_inv[word]))
#             except:
#                 vectory.append(258)
#         X_train.append(vectorx)
#         Y_train.append(vectory)
#     return X_train,Y_train

# def clip_pad_input_sequences(X,Y,min_sentence_length,max_sentence_length): # Doing this for training and validation datasets
#     X_training_clipped = []
#     Y_training_clipped = []
#     for i in range(len(X)):
#         print(i,X[i])
#         if len(X[i]) >= min_sentence_length and len(X[i]) <= max_sentence_length and len(Y[i]) >= min_sentence_length and len(Y[i]) <= max_sentence_length:
#             X_training_clipped.append(torch.Tensor(X[i]))
#             Y_training_clipped.append(torch.Tensor(Y[i]))
#     padded_X_training_clipped = pad_sequence(X_training_clipped,padding_value = vocab_size)
#     padded_Y_training_clipped = pad_sequence(X_training_clipped,padding_value = vocab_size)
#     return padded_X_training_clipped,padded_Y_training_clipped

# def find_closest_embedding(output,vocab,embedded_words): # Need this because we probably need to find the closest embedding to a given embedding and thereby find the word
#     return

# min_sentence_length = 7
# max_sentence_length = 16
# vocab_size = len(vocab)
# # embedded_words = load_glove_embeddings()

# X_train,Y_train = clip_pad_input_sequences(*tokenize_embed(X_train_prelim,Y_train_prelim,),min_sentence_length,max_sentence_length)
# X_valid,Y_valid = clip_pad_input_sequences(*tokenize_embed(X_val_prelim,Y_val_prelim,),min_sentence_length,max_sentence_length)


input_size = 100 # Size of embedding
hidden_size = 256 # Can set this to be a custom value
num_layers = 1 # Can change this
bs = 128
vocab_size = len(vocab)
dropout = 0.2
bidirectional = False
max_sentence_length = 16 # Max number of words in a sentence

class Encoder(nn.Module): # Encoder with a single layer LSTM 
    def __init__(self,max_sentence_length,input_size,hidden_size,dropout,vocab_size,num_layers,bs,bidirectional= False):
        super(Encoder,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.bs = bs
        self.num_layers = num_layers*2 if bidirectional else num_layers
        self.vocab_size = vocab_size
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding(vocab_size,input_size,sparse=False) # Lookup table with word vectors and word index in vocab
#         self.linear = nn.Sequential(nn.Linear(input_size,linear_output_size),nn.Relu(),nn.Linear(linear_output_size,input_size))
        self.lstm_layer = nn.LSTM(input_size, hidden_size, num_layers, batch_first=False,bidirectional = self.bidirectional,dropout = self.dropout) # LSTM moodule
        self.init_weights()
        self.max_sentence_length = max_sentence_length
        
    def forward(self, x, prev_state,indices_given = True):  # X corresponds to the index of the word in the vocabulary (so X can be 2 or 49 etc, and embedding(x) will give the word vector of the 2nd or 49th word in the vocabulary)
        if indices_given:
            embed = self.embedding(x) # Looking up embeddings
        else:
            embed = x.float()
        embed = torch.reshape(torch.Tensor(embed),(self.max_sentence_length,self.bs,self.input_size))
        output, curr_state = self.lstm_layer(embed, prev_state)
        return output,curr_state,embed
    
    def init_weights(self): # Initialising weights
        init.orthogonal_(self.lstm_layer.weight_ih_l0)
        init.uniform_(self.lstm_layer.weight_hh_l0, a=-0.01, b=0.01) # Values taken from the paper
        embedding_weights = torch.FloatTensor(self.vocab_size+1, self.input_size)
        init.uniform_(embedding_weights, a=-0.25, b=0.25) # Values taken from the paper
        for k,v in embedded_words.items():
            embedding_weights[k,:] = torch.FloatTensor(v)
        embedding_weights[0] = torch.FloatTensor([0]*self.input_size)
        embedding_weights[vocab_size] = torch.FloatTensor([0]*self.input_size) # Equivalent embedding for 
        del self.embedding.weight # Deleting any pre-initialised embeddings in nn.embedding.weights and replacing it with our known word embeddings
        self.embedding.weight = nn.Parameter(embedding_weights)
    
    def train(self,x): # Batch feedforward across all training examples
        curr_state = (torch.zeros(self.num_layers,self.bs,self.hidden_size),torch.zeros(self.num_layers,self.bs,self.hidden_size)) # Initialization of hidden and cell state
        for i in range(max(x.shape)//bs):
            if i < max(x.shape)//bs-1:
                wordx = x[:,i:i+bs]
                output, curr_state = encoder.forward(wordx.long(),curr_state)


In [106]:
import numpy as np
import torch.nn as nn
import torch

class Decoder(nn.Module):

    def __init__(self, Encoder, embedding_dims, hidden_size, num_layers = 1):
        super(Decoder, self).__init__()

        self.Encoder = Encoder

        ### QUESTION: What dimension for across q?
        self.softmax = nn.Softmax(dim = -1)                 # or 2?

        # Hyper-parameters
        self.embedding_dims = embedding_dims
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.init_weights(embedding_dims, hidden_size, num_layers)

    def local_loss_contrib(self, q_hat, q_real):
        ### QUESTION: In the paper they have, what looks like, average across text, right?
        sentence_length = q_hat.shape[0]

        
        s = 0
        
        for i in range(q_hat.shape[0]):
            
            dot = torch.dot(torch.log(q_hat[i]), q_real[i])
            s += dot

        return -(1/sentence_length)*s


    def init_weights(self, embedding_dims, hidden_size, num_layers):
        
        # Size of batch = [None, sentence size, embedding dims]  :. size of Wd = [embedding dims, embedding dims] to map to same space
        self.Wd = nn.Linear(in_features = embedding_dims, out_features = embedding_dims, bias = False)
        self.Wv = nn.Linear(in_features = hidden_size, out_features = embedding_dims, bias = False)

        # LSTM init with size [input_size, hidden_size, num_layers]
        self.lstm = nn.LSTM(embedding_dims, hidden_size, num_layers)        
    
    def forward(self, sentence, q_real,indices_given=True):
        """
        A forward pass on the LSTM decoder. Input and ground truth come in as matrix of indicies, later embedded to size [sentence length, batch size, embedding dims]
        """

        # Initialise hidden layer and cell state, size = [num_layers*num_directions, batch, hidden_size]

        h0 = torch.zeros((self.num_layers, sentence.size()[1], self.hidden_size))
        c0 = torch.zeros((self.num_layers, sentence.size()[1], self.hidden_size))

        sentence = sentence.long()
        
        _, (h0, c0),sentence = self.Encoder.forward(sentence, (h0, c0),indices_given)
        q_real = self.Encoder.embedding(q_real.long())

        # dt = Wd * qt = d-weights * previously predicted word
        dt = self.Wd(sentence)

        # h[t+1] = LSTM(dt, ht)          LSTM output paramter = [sentence length, batch, hidden_size*num_directions]
        out, (hn, cn) = self.lstm(dt, (h0, c0))
        
        # Out has dims: [sentence length, batch, embedding_dims]    
        p = self.Wv(out)

        # Softmax p to get q_hat
        q_hat = self.softmax(p)
        q_real_mean = torch.mean(q_real, dim = 1)
        q_hat_mean = torch.mean(q_hat, dim = 1)

        # CROSS ENTROPY TAKEN OVER dim 0 (i.e. sentence length)
        # This is one of the terms at in the L_local summation, which must later be added up and divided
        local_loss = self.local_loss_contrib(q_hat_mean, q_real_mean) /sentence.size()[1]
        
        return local_loss, q_hat,q_real,(h0,c0)

    def train(self, num_epochs, num_iterations, learning_rate):
        self.optimiser = torch.optim.Adam(self.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            for i in range(num_iterations):
                pass ### waiting for encoder output


# def trial_forward_run():
#     embedding_dims = 100
#     hidden_size = 20

#     decoder = Decoder(embedding_dims, hidden_size)

#     inp = torch.randn(5, 3, embedding_dims)
#     gtt = torch.randn(5, 3, embedding_dims)

#     loss, q_hat = decoder.forward(inp, gtt)
#     print(loss)

In [136]:
from itertools import chain
from torchvision.transforms import Normalize
input_size = 100
embedding_dims = 100
hidden_size_enc = 128
hidden_size_dec = 128
num_layers = 1
vocab_size = len(vocab)
dropout = 0.2
max_sentence_length = 16

class Discriminator():
    def __init__(self,encoder):
        super(Discriminator,self).__init__()
        self.encoder = encoder # encoder module
        
    def forward(self,ground_truth_sentence,predicted_sentence,prev_state,indices_given = True): # Prev state is the hidden state and cell state of the encoder at the given timestep
        _,(encoded_hidden_state_gt,encoded_cell_state_gt),_= encoder.forward(ground_truth_sentence.long(),prev_state,indices_given)
        _,(encoded_hidden_state_pred,encoded_cell_state_pred),_ = encoder.forward(predicted_sentence.long(),prev_state,indices_given)
        loss = self.global_loss(encoded_hidden_state_gt,encoded_hidden_state_pred)
        
        return loss
        
    def backward(self): # Have to call loss.backward()
        return 0
    
    def global_loss(self,encoded_gt,encoded_pred): # Have to implement global loss_function
        loss = 0
        for i in range(bs):
            for j in range(bs):
                dotproduct = encoded_pred[:,i,:]*encoded_gt[:,j,:]-encoded_pred[:,i,:]*encoded_gt[:,i,:]
#                 dotproduct = (dotproduct - torch.max(dotproduct))/(torch.max(dotproduct)-torch.min(dotproduct))
                dotproduct = dotproduct.sum(dim=1)
                loss += max(dotproduct[0],0)
        return loss
learning_rate = 0.001
encoder = Encoder(max_sentence_length,input_size,hidden_size,dropout,vocab_size,num_layers,bs,bidirectional= False)
decoder = Decoder(encoder,embedding_dims, hidden_size)
discriminator = Discriminator(encoder)
params = chain(decoder.parameters(),encoder.parameters())
optimizer = torch.optim.RMSprop(decoder.parameters(), lr=learning_rate)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # PyTorch v0.4.0
# model = decoder.to(device)
# print(summary(model,(16,128,100)))
def train(num_epochs,optimizer):
    for epoch in range(num_epochs):
        for i in range(X_train.shape[1]//bs):
            print(i)
            x = X_train[:,i:i+bs]
            y = Y_train[:,i:i+bs]
            local_loss, q_hat,q_real,prev_state = decoder.forward(x, y)
            global_loss = discriminator.forward(q_real,q_hat,prev_state,indices_given=False) 
            total_loss = local_loss + global_loss
            print(local_loss)
            print(total_loss)
            total_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print(decoder.paramters())
train(5,optimizer)

0
tensor(-0.0561, grad_fn=<DivBackward0>)
tensor(7.1333, grad_fn=<AddBackward0>)
1
tensor(-0.1659, grad_fn=<DivBackward0>)
tensor(5011.2700, grad_fn=<AddBackward0>)


KeyboardInterrupt: 